Check Cluster State
====
This notebook check the state of ES cluster and sends mails to all the people substribed to that alert.
It is run once per hour from a cron job.

In [1]:
import subscribers, alerts
import requests

In [2]:
r=requests.get('http://cl-analytics.mwt2.org:9200/_cluster/health')
res=r.json()
print(res)

{'timed_out': False, 'delayed_unassigned_shards': 0, 'initializing_shards': 0, 'number_of_pending_tasks': 0, 'active_primary_shards': 8325, 'unassigned_shards': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'cluster_name': 'atlas', 'active_shards': 16781, 'number_of_data_nodes': 13, 'active_shards_percent_as_number': 100.0, 'status': 'green', 'number_of_nodes': 20, 'relocating_shards': 0}


In [3]:
res['status']='red'

if res['status'] != 'green':
    S = subscribers.subscribers()
    A = alerts.alerts()

if res['status'] == 'red':
    subscribersToRed =  S.getSubscribers('Cluster is in RED')
    for subscriber in subscribersToRed:
        body = 'Dear ' + subscriber[0]+',\n\n'
        body = body + '\tthis mail is to let you that the University of Chicago Elasticsearch cluster is in RED.'
        body = body + '\nBest regards,\nATLAS AAS'
        body = body + '\n\n To change your alerts preferences please you the following link:\n'+subscriber[2]
        print(subscriber)
        A.addAlert(subsriber[1], body)
        
if res['status'] == 'yellow':
    subscribersToYellow = S.getSubscribers('Cluster is in YELLOW')
    for subscriber in subscribersToYellow:
        print(subscriber)
